In [2]:
!pip3 install pmdarima
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics import tsaplots
import pmdarima as pm
import matplotlib.pyplot as plt





     |████████████████████████████████| 921kB 2.8MB/s 


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')



In [4]:
data_path = "/content/drive/My Drive/Colab Notebooks/"

df = pd.read_csv(data_path + "cooked_v3.csv")
print(df.head())
print(df.CalcDateTime.min())
print(df.CalcDateTime.max())

# tickers and their traded volumes
volume_series = df.groupby('Mnemonic')['TradedVolume'].sum()
volume_series.sort_values(ascending=False, inplace=True)
print(volume_series)

df.head()
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True)


          CalcDateTime Mnemonic  MinPrice  MaxPrice  StartPrice  EndPrice  \
0  2017-07-03 08:00:00      SNH     4.450     4.456       4.456     4.450   
1  2017-07-03 08:01:00      SNH     4.450     4.456       4.456     4.450   
2  2017-07-03 08:02:00      SNH     4.450     4.456       4.456     4.450   
3  2017-07-03 08:03:00      SNH     4.457     4.457       4.457     4.457   
4  2017-07-03 08:04:00      SNH     4.457     4.457       4.457     4.457   

   HasTrade  TradedVolume  NumberOfTrades  
0       1.0        5505.0             9.0  
1       0.0           0.0             0.0  
2       0.0           0.0             0.0  
3       1.0         899.0             2.0  
4       0.0           0.0             0.0  
2017-07-03 08:00:00
2018-05-15 20:00:00
Mnemonic
SNH     4.182139e+09
DBK     2.346360e+09
EOAN    1.883320e+09
DTE     1.695683e+09
CBK     1.575910e+09
RWE     8.374443e+08
IFX     7.630757e+08
SVAB    6.026515e+08
LHA     5.934044e+08
DAI     4.894308e+08
O2D     4.2069

In [0]:

for ticker in volume_series.index:
    start_time = datetime.now()
    ticker_data = df[df['Mnemonic'] == ticker]
    ticker_data.reset_index(inplace=True)

    X = pd.to_numeric(pd.to_datetime(ticker_data['CalcDateTime']))
    X = X.values.reshape(-1, 1)

    Y = ticker_data['StartPrice']
    print("Ticker " + ticker + " has " + str(len(X)) + " records.")


    ticker_data['CalcDateTime'] = pd.to_datetime(ticker_data['CalcDateTime'])
    #ticker_data = ticker_data[['CalcDateTime', 'StartPrice']]
    ticker_data.set_index(ticker_data['CalcDateTime'], inplace=True)
    
    exclude = ['AT1']

#######################################################
#   TIME SERIES ANALYSIS IN PYTHON
#######################################################

    # decomp freq: https://www.cbcity.de/timeseries-decomposition-in-python-with-statsmodels-and-pandas
#     result = seasonal_decompose(ticker_data.StartPrice, freq=10080)
#     trend = result.trend
#     result.plot()
#     plt.show()
    
#     # ACF Plots
#     pm.plot_acf(ticker_data.StartPrice[0:49])
#     # print(pm.utils.acf(ticker_data.StartPrice[0:49]))
    
#     fig = tsaplots.plot_acf(ticker_data.StartPrice, lags = 20, zero=False)
#     ax = fig.add_subplot(111)
#     ax.set_xlabel('Lags')
#     ax.set_ylabel('Autocorrelation')
#     ax.set_title('Autocorrelation plot for ' + ticker + " (50 lags)" )
#     plt.show()
    
#     # PACF Plots
#     pm.plot_pacf(ticker_data.StartPrice[0:49])
#     # print(pm.utils.pacf(ticker_data.StartPrice[0:49]))
    
#     fig = tsaplots.plot_pacf(ticker_data.StartPrice, lags = 20, zero=False)
#     ax = fig.add_subplot(111)
#     ax.set_xlabel('Lags')
#     ax.set_ylabel('Partial Autocorrelation')
#     ax.set_title('Partial Autocorrelation plot for ' + ticker + " (50 lags)" )
#     plt.show()
    
#     # adfuller test
#     if ticker not in exclude:
#       results = adfuller(ticker_data.StartPrice)
#       print(results)
    
    
#######################################################
#   ROLLING AVERAGES IN PYTHON
#######################################################
#     # Normal Plot
#     daily_avg = ticker_data.StartPrice
    
#     ax = daily_avg.plot()
#     ax.set_xlabel('Date')
#     ax.set_ylabel('Minutely Start Price')
#     ax.set_title('Minutely Rolling Mean of Starting Prices for ' + ticker)
#     plt.show()
    
#     # Daily Rolling Averages
#     daily_avg = ticker_data.StartPrice.rolling(window =60*24).mean()
    
#     ax = daily_avg.plot()
#     ax.set_xlabel('Date')
#     ax.set_ylabel('Daily Start Price')
#     ax.set_title('Daily Rolling Mean of Starting Prices for ' + ticker)
#     plt.show()
    
#     # Weekly Rolling Averages
#     weekly_avg = ticker_data.StartPrice.rolling(window =60*24*7).mean()
    
#     ax = weekly_avg.plot()
#     ax.set_xlabel('Date')
#     ax.set_ylabel('Weekly Start Price')
#     ax.set_title('Weekly Rolling Mean of Starting Prices for ' + ticker)
#     plt.show()
    
    
    if ticker not in exclude:
      # predict last 15 values since arima on 25% of the data is taking too long
      train_index = np.arange(0,len(X) - 15)
      test_index = np.arange(len(X) - 15, len(X))
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = Y[train_index], Y[test_index]

      stepwise_model = pm.auto_arima(ticker_data.StartPrice)
      print(stepwise_model.summary())
      print(stepwise_model.aic())



    end_time = datetime.now()
    c = end_time - start_time
    print("Generated Rolling Means in:" + str(c.total_seconds()))


Ticker SNH has 156457 records.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:               156457
Model:               SARIMAX(3, 1, 3)   Log Likelihood              601849.530
Date:                Sat, 04 May 2019   AIC                       -1203683.060
Time:                        21:00:50   BIC                       -1203603.376
Sample:                             0   HQIC                      -1203659.354
                             - 156457                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -1.209e-05   6.95e-06     -1.741      0.082   -2.57e-05    1.52e-06
ar.L1          0.1675      0.005     31.357      0.000       0.157       0.178
ar.L2          0.7361      0.004    202.993      0.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/usr/local/lib/python3.6/dist-packages/stats

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:               156457
Model:               SARIMAX(0, 1, 3)   Log Likelihood              506577.954
Date:                Sat, 04 May 2019   AIC                       -1013145.907
Time:                        21:08:55   BIC                       -1013096.105
Sample:                             0   HQIC                      -1013131.091
                             - 156457                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -2.879e-05    2.4e-05     -1.199      0.231   -7.59e-05    1.83e-05
ma.L1         -0.0129      0.001    -11.635      0.000      -0.015      -0.011
ma.L2          0.0080      0.002      5.135      0.0

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
/usr/local/lib/python3.6/dist-packages/stats

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:               156446
Model:               SARIMAX(2, 1, 2)   Log Likelihood              973905.628
Date:                Sat, 04 May 2019   AIC                       -1947799.257
Time:                        21:48:12   BIC                       -1947739.494
Sample:                             0   HQIC                      -1947781.478
                             - 156446                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept  -2.551e-07   1.28e-07     -1.999      0.046   -5.05e-07   -5.04e-09
ar.L1          0.4099      0.000    907.012      0.000       0.409       0.411
ar.L2          0.4185      0.001    670.714      0.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pmdarima/arima/_auto_solvers.py:207: ModelFitWarning: Unable to fit ARIMA for order=(2, 1, 2) seasonal_order=(0, 0, 0, 1); data is likely non-stationary. (if you do not want to see these warnings, run with error_action="ignore")
  ModelFitWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matr

                           Statespace Model Results                           
Dep. Variable:                      y   No. Observations:               156457
Model:               SARIMAX(0, 1, 1)   Log Likelihood              407860.188
Date:                Sat, 04 May 2019   AIC                        -815714.377
Time:                        21:50:05   BIC                        -815684.495
Sample:                             0   HQIC                       -815705.487
                             - 156457                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   2.902e-05   4.72e-05      0.615      0.538   -6.35e-05       0.000
ma.L1          0.0053      0.001      4.947      0.000       0.003       0.007
sigma2         0.0003   6.38e-08   4996.225      0.0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
